In [5]:
import os
import urllib.request
import pandas as pd
import json
import ssl

client_id = "VVrSb7DXLYUr2IRlQNSU"
client_secret = "pJSQFrP13Z"
encText = urllib.parse.quote("미국")

# 모든 페이지에 대해서 데이터 수집 및 엑셀 파일에 기록
for page in range(1, 11):
    url = f"https://openapi.naver.com/v1/search/book?query={encText}&display=100&start={100*(page-1)+1}"
    
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    context = ssl._create_unverified_context()
    response = urllib.request.urlopen(request, context=context)
    rescode = response.getcode()

    if(rescode==200):
        response_body = response.read()
        result = response_body.decode('utf-8')
        
        book_result = json.loads(result)
        
        df = pd.DataFrame(book_result['items'])

        read_data = pd.read_csv("naver_book.csv", sep=",")
        print (read_data)
        df.to_csv("naver_book.csv")

        print(f"페이지 {page} 책 데이터 수집 완료")
            
    else:
        print(f"페이지 {page} 책 데이터 수집 실패")


print("모든 페이지의 책 데이터 수집 완료")


    Unnamed: 0                                         title  \
0            0   지금 알려줄게요 미국대학원 (미국 석박사과정 장학금 받고 1년 만에 합격하기)   
1            1                        제인 에어 (미국대학위원회 추천 필독서)   
2            2                       오만과 편견 (미국대학위원회 추천 필독서)   
3            3                      위대한 개츠비 (미국대학위원회 추천 필독서)   
4            4  미국 유학 100문 100답 (전문 컨설턴트가 말하는 미국 대학 가기 프로젝트)   
..         ...                                           ...   
95          95                        위기의 미국 의회 (개혁정치의 허와 실)   
96          96                                      미국의 산업상담   
97          97                                      미국 예산정치론   
98          98                                      미국희곡의 이해   
99          99                                      미국의 지방자치   

                                                 link  \
0   https://search.shopping.naver.com/book/catalog...   
1   https://search.shopping.naver.com/book/catalog...   
2   https://search.shopping.naver.com/book/catalog...   
3  

In [10]:
import os
import urllib.request
import pandas as pd
import json
import ssl

client_id = "VVrSb7DXLYUr2IRlQNSU"
client_secret = "pJSQFrP13Z"
encText = urllib.parse.quote("미국")

# 빈 데이터프레임 생성
df = pd.DataFrame()

# 모든 페이지에 대해서 데이터 수집
for page in range(1, 11):
    url = f"https://openapi.naver.com/v1/search/book?query={encText}&display=100&start={100*(page-1)+1}"
    
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    context = ssl._create_unverified_context()
    response = urllib.request.urlopen(request, context=context)
    rescode = response.getcode()

    if(rescode==200):
        response_body = response.read()
        result = response_body.decode('utf-8')
        
        book_result = json.loads(result)
        
        # 현재 페이지에서 수집한 데이터를 데이터프레임에 추가
        df_page = pd.DataFrame(book_result['items'])
        df = pd.concat([df, df_page], axis=0)
        

        print(f"페이지 {page} 책 데이터 수집 완료")
            
    else:
        print(f"페이지 {page} 책 데이터 수집 실패")

# 데이터프레임을 csv 파일에 저장
df.to_csv("naver_book.csv", index=True)

read_data = pd.read_csv("naver_book.csv", sep=",")
print (read_data)

print("모든 페이지의 책 데이터 수집 완료")


페이지 1 책 데이터 수집 완료
페이지 2 책 데이터 수집 완료
페이지 3 책 데이터 수집 완료
페이지 4 책 데이터 수집 완료
페이지 5 책 데이터 수집 완료
페이지 6 책 데이터 수집 완료
페이지 7 책 데이터 수집 완료
페이지 8 책 데이터 수집 완료
페이지 9 책 데이터 수집 완료
페이지 10 책 데이터 수집 완료
     Unnamed: 0                                     title  \
0             0  프렌즈 미국 동부(2022~2023) (뉴욕·워싱턴 DC·보스턴·시카고)   
1             1       뉴욕 셀프트래블(2023-2024) (2023-2024 최신판)   
2             2                               다시, 서쪽으로 가다   
3             3   프렌즈 캐나다(2022~2023) (벤쿠버·토론토·몬트리올·퀘벡·로키)   
4             4  미국 서부 셀프트래블(2022-2023) (믿고 보는 해외여행 가이드북)   
..          ...                                       ...   
995          95                    위기의 미국 의회 (개혁정치의 허와 실)   
996          96                                  미국의 산업상담   
997          97                                  미국 예산정치론   
998          98                                  미국희곡의 이해   
999          99                                  미국의 지방자치   

                                                  link  \
0    https://search.shopping

.csv로 저장하면 가독성이 너무 떨어지고, .csv를 vscode가 아닌 엑셀로 열었을 때 글자가 깨져 아래는 .xlsx로 만드는 코드입니다.

In [4]:
import os
import urllib.request
import json
import ssl
import xlsxwriter

client_id = "VVrSb7DXLYUr2IRlQNSU"
client_secret = "pJSQFrP13Z"
encText = urllib.parse.quote("미국")

# 엑셀 파일 생성
book_workbook = xlsxwriter.Workbook('naver_book.xlsx')
book_worksheet = book_workbook.add_worksheet()

# 엑셀에 헤더 쓰기
headers = ['title', 'link', 'author', 'price', 'discount', 'publisher', 'pubdate', 'isbn', 'description']
for col, header in enumerate(headers):
    book_worksheet.write(0, col, header)

# 모든 페이지에 대해서 데이터 수집 및 엑셀 파일에 기록
for page in range(1, 11):
    url = f"https://openapi.naver.com/v1/search/book?query={encText}&display=100&start={100*(page-1)+1}"
    
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    context = ssl._create_unverified_context()
    response = urllib.request.urlopen(request, context=context)
    rescode = response.getcode()

    if(rescode==200):
        response_body = response.read()
        result = json.loads(response_body)
        book_list = result['items']

        # 엑셀에 데이터 쓰기
        for i, book in enumerate(book_list, 1+100*(page-1)):
            book_worksheet.write(i, 0, book['title'])
            book_worksheet.write(i, 1, book['link'])
            book_worksheet.write(i, 2, book['author'])
            if 'price' in book:
                book_worksheet.write(i, 3, book['price'])
            else:
                book_worksheet.write(i, 3, "")
            book_worksheet.write(i, 4, book['discount'])
            book_worksheet.write(i, 5, book['publisher'])
            book_worksheet.write(i, 6, book['pubdate'])
            book_worksheet.write(i, 7, book['isbn'])
            book_worksheet.write(i, 8, book['description'])

        print(f"페이지 {page} 책 데이터 수집 완료")
            
    else:
        print(f"페이지 {page} 책 데이터 수집 실패")

# 엑셀 파일 저장
book_workbook.close()
print("모든 페이지의 책 데이터 수집 완료")


페이지 1 책 데이터 수집 완료
페이지 2 책 데이터 수집 완료
페이지 3 책 데이터 수집 완료
페이지 4 책 데이터 수집 완료
페이지 5 책 데이터 수집 완료
페이지 6 책 데이터 수집 완료
페이지 7 책 데이터 수집 완료
페이지 8 책 데이터 수집 완료
페이지 9 책 데이터 수집 완료
페이지 10 책 데이터 수집 완료
모든 페이지의 책 데이터 수집 완료
